# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [4]:
 data = spark.read.csv('/FileStore/tables/hack_data.csv',header=True, inferSchema=True)

In [5]:
data.toPandas().head()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
0,8.0,391.09,1,2.96,7.0,Slovenia,72.37
1,20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08
2,31.0,356.32,1,3.71,8.0,Tokelau,70.58
3,2.0,228.08,1,2.48,8.0,Bolivia,70.80
4,20.0,408.50,0,3.57,8.0,Iraq,71.28


In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

Out[6]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
#Creamos un objeto assembler para dejar los datos en el formato aceptado por las librerías de pyspark
#Dejaremos fuera de las variables a la variable location
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'], outputCol='features')

In [9]:
final_data = assembler.transform(data)

In [10]:
final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

Aplicaremos StandardScaler para que las features tengan datos de orden similar.

In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
#Creamos objeto scaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [14]:
scaler_model = scaler.fit(final_data)

In [15]:
cluster_final_data = scaler_model.transform(final_data)

In [16]:
cluster_final_data.toPandas().head()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed,features,scaledFeatures
0,8.0,391.09,1,2.96,7.0,Slovenia,72.37,"[8.0, 391.09, 1.0, 2.96, 7.0, 72.37]","[0.5678510846650524, 1.3658432518957642, 1.997..."
1,20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08,"[20.0, 720.99, 0.0, 3.04, 9.0, 69.08]","[1.419627711662631, 2.517986463945197, 0.0, 1...."
2,31.0,356.32,1,3.71,8.0,Tokelau,70.58,"[31.0, 356.32, 1.0, 3.71, 8.0, 70.58]","[2.2004229530770782, 1.2444124562517545, 1.997..."
3,2.0,228.08,1,2.48,8.0,Bolivia,70.80,"[2.0, 228.08, 1.0, 2.48, 8.0, 70.8]","[0.1419627711662631, 0.7965469045293562, 1.997..."
4,20.0,408.50,0,3.57,8.0,Iraq,71.28,"[20.0, 408.5, 0.0, 3.57, 8.0, 71.28]","[1.419627711662631, 1.4266459597520256, 0.0, 1..."


Vamos a entrenar dos modelos, uno con K=2 y otro con K=3 para ver si la cantidad de elementos en cada grupo es similar, con lo cual podremos hacernos una idea de si había 2 o 3 hackers, según la información de que los hackers hacen un trade off de sus ataques.

In [18]:
from pyspark.ml.clustering import KMeans

In [19]:
#Con .setSeed(1) podremos replicar el experimento
kmeans2clust = KMeans(featuresCol='scaledFeatures', k=2).setSeed(1)
kmeans3clust = KMeans(featuresCol='scaledFeatures', k=3).setSeed(1)

In [20]:
model2clust = kmeans2clust.fit(cluster_final_data)
model3clust = kmeans3clust.fit(cluster_final_data)

In [21]:
#WSSSE: Within set sum of squared error
print('WSSSE modelos con 2 clusters')
print(model2clust.computeCost(cluster_final_data))
print('WSSSE modelos con 3 clusters')
print(model3clust.computeCost(cluster_final_data))

WSSSE modelos con 2 clusters
601.7707512676716
WSSSE modelos con 3 clusters
434.75507308487647

In [22]:
centers2clust = model2clust.clusterCenters()
centers3clust = model3clust.clusterCenters()

In [23]:
centers2clust

Out[70]: [array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
 5.26676612]),
 array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
 3.28474 ])]

In [24]:
centers3clust

Out[71]: [array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
 5.26676612]),
 array([3.05623261, 2.95754486, 1.99757683, 3.2079628 , 4.49941976,
 3.26738378]),
 array([2.93719177, 2.88492202, 0. , 3.19938371, 4.52857793,
 3.30407351])]

In [25]:
#Predicción de clusters
model2clust.transform(cluster_final_data).toPandas().head()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed,features,scaledFeatures,prediction
0,8.0,391.09,1,2.96,7.0,Slovenia,72.37,"[8.0, 391.09, 1.0, 2.96, 7.0, 72.37]","[0.5678510846650524, 1.3658432518957642, 1.997...",0
1,20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08,"[20.0, 720.99, 0.0, 3.04, 9.0, 69.08]","[1.419627711662631, 2.517986463945197, 0.0, 1....",0
2,31.0,356.32,1,3.71,8.0,Tokelau,70.58,"[31.0, 356.32, 1.0, 3.71, 8.0, 70.58]","[2.2004229530770782, 1.2444124562517545, 1.997...",0
3,2.0,228.08,1,2.48,8.0,Bolivia,70.80,"[2.0, 228.08, 1.0, 2.48, 8.0, 70.8]","[0.1419627711662631, 0.7965469045293562, 1.997...",0
4,20.0,408.50,0,3.57,8.0,Iraq,71.28,"[20.0, 408.5, 0.0, 3.57, 8.0, 71.28]","[1.419627711662631, 1.4266459597520256, 0.0, 1...",0


In [26]:
model3clust.transform(final_data).toPandas().head()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed,features,scaledFeatures,prediction
0,8.0,391.09,1,2.96,7.0,Slovenia,72.37,"[8.0, 391.09, 1.0, 2.96, 7.0, 72.37]","[0.5678510846650524, 1.3658432518957642, 1.997...",0
1,20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08,"[20.0, 720.99, 0.0, 3.04, 9.0, 69.08]","[1.419627711662631, 2.517986463945197, 0.0, 1....",0
2,31.0,356.32,1,3.71,8.0,Tokelau,70.58,"[31.0, 356.32, 1.0, 3.71, 8.0, 70.58]","[2.2004229530770782, 1.2444124562517545, 1.997...",0
3,2.0,228.08,1,2.48,8.0,Bolivia,70.80,"[2.0, 228.08, 1.0, 2.48, 8.0, 70.8]","[0.1419627711662631, 0.7965469045293562, 1.997...",0
4,20.0,408.50,0,3.57,8.0,Iraq,71.28,"[20.0, 408.5, 0.0, 3.57, 8.0, 71.28]","[1.419627711662631, 1.4266459597520256, 0.0, 1...",0


Contaremos la cantidad de elementos en cada cluster

In [28]:
predictions2clust = model2clust.transform(cluster_final_data)
predictions3clust = model3clust.transform(cluster_final_data)

In [29]:
predictions2clust.groupBy('prediction').count().toPandas().head()

,prediction,count
0,1,167
1,0,167


Para el caso de 2 clusters vemos que la cantidad de elementos en cada grupo es igual, por lo que es posible pensar que hay 2 hackers, veremos qué pasas con el caso de 3 clusters.

In [31]:
predictions3clust.groupBy('prediction').count().toPandas().head()

,prediction,count
0,1,88
1,2,79
2,0,167


Cuando buscamos 3 grupos vemos que hay dos grupos con una cantidad similar de elementos pero el tercer grupo tiene una cantidad mucho mayor, al parecer un grupo se dividió en dos y el otro permanenció con la cantidad de elementos original. Dado lo anterior, podemos aceptar que hay 2 hackers.